In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
data = pd.read_csv("/content/zameen.csv")
data.head()

,location,city,baths,bedrooms,date_added,Area Size,price
0,G-10,Islamabad,2,2,2/4/2019,4.0,10000000
1,E-11,Islamabad,3,3,5/4/2019,5.6,6900000
2,G-15,Islamabad,6,5,7/17/2019,8.0,16500000
3,Bani Gala,Islamabad,4,4,4/5/2019,2.0,43500000
4,DHA Defence,Islamabad,3,3,7/10/2019,8.0,7000000


In [ ]:
data.shape

(42775, 7)

In [ ]:
data['Area Size']=data['Area Size'].astype(int)

In [ ]:
data=data[~data['Area Size'].isna()]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42775 entries, 0 to 42774
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   location    42775 non-null  object
 1   city        42775 non-null  object
 2   baths       42775 non-null  int64 
 3   bedrooms    42775 non-null  int64 
 4   date_added  42775 non-null  object
 5   Area Size   42775 non-null  int64 
 6   price       42775 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 2.6+ MB


In [ ]:
data.describe(include='all')

,location,city,baths,bedrooms,date_added,Area Size,price
count,42775,42775,42775.000000,42775.000000,42775,42775.000000,4.277500e+04
unique,1299,5,NaN,NaN,124,NaN,NaN
top,DHA Defence,Karachi,NaN,NaN,7/3/2019,NaN,NaN
freq,5078,15020,NaN,NaN,2459,NaN,NaN
mean,NaN,NaN,3.213442,3.330076,NaN,5.661695,1.870825e+07
std,NaN,NaN,2.225710,2.037399,NaN,4.947693,3.812862e+07
min,NaN,NaN,0.000000,0.000000,NaN,0.000000,1.000000e+00
25%,NaN,NaN,2.000000,2.000000,NaN,3.000000,2.275000e+05
50%,NaN,NaN,3.000000,3.000000,NaN,5.000000,8.000000e+06
75%,NaN,NaN,5.000000,5.000000,NaN,8.000000,2.000000e+07


In [ ]:
x=data.drop(['date_added','price'],axis=1)
y = data['price']

In [ ]:
x

,location,city,baths,bedrooms,Area Size
0,G-10,Islamabad,2,2,4
1,E-11,Islamabad,3,3,5
2,G-15,Islamabad,6,5,8
3,Bani Gala,Islamabad,4,4,2
4,DHA Defence,Islamabad,3,3,8
...,...,...,...,...,...
42770,Satiana Road,Faisalabad,0,0,8
42771,Satiana Road,Faisalabad,0,0,4
42772,Satiana Road,Faisalabad,0,0,12
42773,Satiana Road,Faisalabad,0,0,10


In [ ]:
y

0        10000000
1         6900000
2        16500000
3        43500000
4         7000000
           ...   
42770       25000
42771       15000
42772       25000
42773       25000
42774       12000
Name: price, Length: 42775, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
ohe=OneHotEncoder()
ohe.fit(x[['location','city']])

OneHotEncoder()

In [ ]:
ohe.categories_

[array(['12th Avenue', '204 Chak Road', '7th Avenue', ..., 'Zubair Colony',
        'Zulfiqar Colony', 'circular road'], dtype=object),
 array(['Faisalabad', 'Islamabad', 'Karachi', 'Lahore', 'Rawalpindi'],
       dtype=object)]

In [ ]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['location','city']),
                                    remainder='passthrough')

In [ ]:
lr=LinearRegression()

In [ ]:
ohe = OneHotEncoder(handle_unknown = 'ignore')

In [ ]:
pipe=make_pipeline(column_trans,lr)

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['12th Avenue', '204 Chak Road', '7th Avenue', ..., 'Zubair Colony',
       'Zulfiqar Colony', 'circular road'], dtype=object),
                                                                            array(['Faisalabad', 'Islamabad', 'Karachi', 'Lahore', 'Rawalpindi'],
      dtype=object)]),
                                                  ['location', 'city'])])),
                ('linearregression', LinearRegression())])

In [ ]:
y_pred=pipe.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

0.21066525034151318

In [ ]:
scores=[]
for i in range(15):
    x_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [ ]:
np.argmax(scores)

6

In [ ]:
scores[np.argmax(scores)]

0.26837421450125687

In [ ]:
import pickle

In [ ]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [ ]:
pipe.predict(pd.DataFrame(columns=['location','city','baths','bedrooms','Area Size'],data=np.array(['G-10','Islamabad',2,2,4.0]).reshape(1,5)))

array([7668560.16395582])

In [ ]:
pipe.steps[0][1].transformers[0][1].categories[0]

array(['12th Avenue', '204 Chak Road', '7th Avenue', ..., 'Zubair Colony',
       'Zulfiqar Colony', 'circular road'], dtype=object)

In [ ]:
data.to_csv('Cleaned_data.csv')